# AWS Costs

In [1]:
import os
import boto3
import json
import pandas as pd
from time import time
from datetime import datetime, timedelta

Set the name of your AWS profile. This defines the account that you wish to retrieve metrics from. This can be different to the account that you logged into.</br>__Note:__ prior to executing this notebook, you must have logged into AWS. You must have done this within the same environment as the notebook is running. If you are not sure, open a JupyterLabs __Terminal__ window and login to AWS there.

In [2]:
AWS_PROFILE_NAME = '<REPLACE WITH YOUR AWS PROFILE NAME>'

__Import configuration overrides__ from local file <code>configuration.py</code>. If it exists, this will import values from an external configuration file that will override those set previously in this notebook. </br>__Note:__ the Kernel must be restarted after changes to the <code>configuration.py</code>. Otherwise the changes will not be imported into the notebook.

In [3]:
# Import overrides 
if os.path.isfile('./configuration.py'): 
    import configuration as cfg
    # Configure the overrides
    AWS_PROFILE_NAME = cfg.AWS_PROFILE_NAME

In [4]:
session = boto3.Session(profile_name = AWS_PROFILE_NAME)
client  = session.client('ce')

Enter the dimensions used to retrieve the metrics. </br>__Note:__ normally you will want to use the default setting.

In [5]:
# Specify the group-by dimensions (e.g., 'SERVICE' or 'LINKED_ACCOUNT')
group_by_dimensions = [{'Type': 'DIMENSION', 'Key': 'SERVICE'}]

In [6]:
class AWSQueryStartDateTooOld(Exception): pass

def validateStartDate(StartTime:datetime = 0):
    # Compare the StartDate value against the validation rules. 
    # Raise an error if it is not valid
    if StartTime < datetime.now() - timedelta(days = 14): raise AWSQueryStartDateTooOld('When using the HOURLY Granularity. The StartTime can be no older than 14 days.')

### Cost Metric Explanation
__Amortized:__ A cost metric that represents the effective daily rate of upfront, prepaid fees, combined with monthly reservation fees, including applied discount rates, and spread across the billing period.</br>
__Unblended:__ This cost metric displays the amount charged at the time of usage.</br>
__Blended:__ This cost metric is based on the average rate for each instance type for all member accounts of an organization.

In [7]:
def getServiceCosts(StartTime:datetime = datetime.now() - timedelta(days = 1), 
                    EndTime:datetime   = datetime.now(),
                    CostMetric:str     = 'BlendedCost',
                    Granularity:str    = 'DAILY'):       # MONTHLY, DAILY or HOURLY
    
    # Validate StartTime
    validateStartDate(StartTime = StartTime)
    
    # Construct dataframes for charge period records and all combined costs
    cost_df = pd.DataFrame()
    rec_df  = pd.DataFrame()
    
    # The format of the timestamps is different when the Granularity is set to HOURLY
    DATE_FORMATTER = '%Y-%m-%dT%H:%M:%SZ' if Granularity == 'HOURLY' else '%Y-%m-%d'
    
    # Retrieve the data from the AWS API
    cost_data = client.get_cost_and_usage(TimePeriod  = {'Start' : StartTime.strftime(DATE_FORMATTER), 
                                                         'End'   : EndTime.strftime(DATE_FORMATTER)},
                                          Granularity =  Granularity,
                                          Metrics     = [CostMetric],
                                          GroupBy     = [{'Type' : 'DIMENSION', 
                                                          'Key'  : 'SERVICE'}],
                                          )['ResultsByTime']
    
    # Populate the record dataframe with the values of a single charge period
    for period_data in cost_data:
        startDate = pd.to_datetime(period_data['TimePeriod']['Start'])
        endDate   = pd.to_datetime(period_data['TimePeriod']['End'  ])
        for service in period_data['Groups']:
            rec_df['Service'  ] = service['Keys']
            rec_df['StartDate'] = startDate
            rec_df['EndDate'  ] = endDate
            rec_df['Amount'   ] = service['Metrics']['BlendedCost']['Amount']
            rec_df['Unit'     ] = service['Metrics']['BlendedCost']['Unit']
            # Append the record dataframe to the combined costs dataframe
            cost_df = pd.concat([cost_df, rec_df]).drop_duplicates().reset_index(drop=True)

    return cost_df    

In [8]:
getServiceCosts()

,Service,StartDate,EndDate,Amount,Unit
0,AWS CloudTrail,2023-11-18,2023-11-19,0,USD
1,AWS Cost Explorer,2023-11-18,2023-11-19,0.56,USD
2,AWS Key Management Service,2023-11-18,2023-11-19,0.0000059982,USD
3,AWS Secrets Manager,2023-11-18,2023-11-19,0.000005,USD
4,AWS Security Hub,2023-11-18,2023-11-19,0.0519564648,USD
5,EC2 - Other,2023-11-18,2023-11-19,0.0351111116,USD
6,Amazon GuardDuty,2023-11-18,2023-11-19,0.0223944,USD
7,Amazon Simple Notification Service,2023-11-18,2023-11-19,0.000000475,USD
8,Amazon Simple Queue Service,2023-11-18,2023-11-19,0.0000004,USD
9,Amazon Simple Storage Service,2023-11-18,2023-11-19,0.0000379225,USD


In [9]:
getServiceCosts(StartTime   = datetime(2023, 11, 10, 1, 1), 
                EndTime     = datetime(2023, 11, 11, 4, 1),
                CostMetric  = 'BlendedCost',
                Granularity = 'HOURLY')

,Service,StartDate,EndDate,Amount,Unit
0,AWS CloudTrail,2023-11-10 01:00:00+00:00,2023-11-10 02:00:00+00:00,0.00582,USD
1,AWS Security Hub,2023-11-10 01:00:00+00:00,2023-11-10 02:00:00+00:00,0.0002335824,USD
2,EC2 - Other,2023-11-10 01:00:00+00:00,2023-11-10 02:00:00+00:00,0.0023266667,USD
3,Amazon GuardDuty,2023-11-10 01:00:00+00:00,2023-11-10 02:00:00+00:00,0.000958,USD
4,AWS CloudTrail,2023-11-10 02:00:00+00:00,2023-11-10 03:00:00+00:00,0.00664,USD
...,...,...,...,...,...
114,Amazon GuardDuty,2023-11-11 02:00:00+00:00,2023-11-11 03:00:00+00:00,0.0009615,USD
115,AWS CloudTrail,2023-11-11 03:00:00+00:00,2023-11-11 04:00:00+00:00,0,USD
116,AWS Security Hub,2023-11-11 03:00:00+00:00,2023-11-11 04:00:00+00:00,0.0002335824,USD
117,EC2 - Other,2023-11-11 03:00:00+00:00,2023-11-11 04:00:00+00:00,0.0023266667,USD
